In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow.lite as tflite
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Question 1

In [2]:
model = keras.models.load_model("dogs_cats_10_0.687.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open("dogs_cats_10_0.687.tflite", "wb") as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\hamed\AppData\Local\Temp\tmp8gszy9ms\assets


## Question 2

In [4]:
interpreter = tflite.Interpreter(model_path="dogs_cats_10_0.687.tflite")
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

print(input_index)
print(output_index)

0
13


In [5]:
# Preparing the Image

from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [6]:
url = "https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg"
downloaded_image = download_image(url)
prepared_image = prepare_image(downloaded_image, (150,150))

## Question 3

In [8]:
x = np.array(prepared_image, dtype="float32")
 x = np.array([x])

datagen = ImageDataGenerator(rescale=1./255)
X = datagen.flow(x, batch_size=1)
X = X.next()

X[0][0][0]

array([0.7058824, 0.7725491, 0.7490196], dtype=float32)

## Question 4

In [9]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)
preds[0].tolist()[0]

0.7704913020133972